# High-Performance Loan Risk Prediction (GPU Accelerated)

**Hardware:** Designed for NVIDIA GPUs (RTX 30xx/40xx series recommended).

### Architectures:
1.  **Deep Residual Neural Network (ResNet-MLP)**: Uses residual skip connections, Batch Normalization, and GELU activations. Optimized with `OneCycleLR` scheduler.
2.  **High-Fidelity Approximation SVM**: Uses a high-dimensional Nystroem feature map (`n_components=2500`) to closely mimic a true RBF kernel, solved via Linear SVM.

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import OneCycleLR

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.kernel_approximation import Nystroem
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.calibration import CalibratedClassifierCV

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚡ Processing Unit: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"   Device Tensor: {device}")

⚡ Processing Unit: NVIDIA GeForce RTX 4060 Laptop GPU
   Device Tensor: cuda


## 1. Data Pipeline

In [2]:
train_path = 'finance/train_updated.csv'
test_path = 'finance/test_updated.csv'

try:
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
except FileNotFoundError:
    print("❌ Files not found. Ensure 'finance/train_updated.csv' exists.")
    raise

# --- Preprocessing Setup ---
target = 'RiskFlag'
id_col = 'ProfileID'

X = train_df.drop([target, id_col], axis=1)
y = train_df[target]
X_test_submission = test_df.drop([id_col], axis=1)
test_ids = test_df[id_col]

# 1. Binary Mapping
binary_cols = ['OwnsProperty', 'FamilyObligation', 'JointApplicant']
for col in binary_cols:
    X[col] = X[col].map({'Yes': 1, 'No': 0})
    X_test_submission[col] = X_test_submission[col].map({'Yes': 1, 'No': 0})

# 2. Ordinal Encoding (Preserving Rank Information)
qualification_order = ["High School", "Bachelor's", "Master's", "PhD"]
workcategory_order = ["Unemployed", "Part-time", "Full-time", "Self-employed"]

ordinal_cols = ['QualificationLevel', 'WorkCategory']
ordinal_encoder = OrdinalEncoder(categories=[qualification_order, workcategory_order], 
                                 handle_unknown='use_encoded_value', unknown_value=-1)

X[ordinal_cols] = ordinal_encoder.fit_transform(X[ordinal_cols])
X_test_submission[ordinal_cols] = ordinal_encoder.transform(X_test_submission[ordinal_cols])

# 3. Full Transformation Pipeline
nominal_cols = ['RelationshipStatus', 'FundUseCase']
numerical_cols = [c for c in X.columns if c not in ordinal_cols + nominal_cols + binary_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('nom', OneHotEncoder(handle_unknown='ignore', sparse_output=False), nominal_cols),
        ('ord', 'passthrough', ordinal_cols),
        ('bin', 'passthrough', binary_cols)
    ]
)

X_processed = preprocessor.fit_transform(X)
X_test_processed = preprocessor.transform(X_test_submission)

X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.15, stratify=y, random_state=42)

print(f"✅ Data Processed. Input Dimensions: {X_train.shape[1]} features.")

✅ Data Processed. Input Dimensions: 22 features.


## 2. High-Fidelity SVM (Nystroem Approximation)

We scale up `n_components` to 2500. This creates a very rich feature map that mimics the non-linear RBF kernel almost perfectly, allowing the LinearSVC to find complex decision boundaries.

In [3]:
print("⏳ Training High-Fidelity SVM... (This uses CPU but is optimized)")

# 1. Feature Map: Projects data into 2500 dimensions to approximate RBF kernel
feature_map = Nystroem(kernel='rbf', gamma=0.1, n_components=2500, random_state=42)

# 2. Classifier: Linear SVM on the projected features
svm_clf = LinearSVC(dual=False, C=1.0, class_weight='balanced', max_iter=2000, random_state=42)

# 3. CalibratedClassifierCV: Wraps SVM to give us actual probabilities (predict_proba) instead of just labels
svm_pipeline = Pipeline([
    ('feature_map', feature_map),
    ('svm_calibrated', CalibratedClassifierCV(svm_clf, cv=3)) 
])

svm_pipeline.fit(X_train, y_train)

# Validation
y_val_probs_svm = svm_pipeline.predict_proba(X_val)[:, 1]
auc_score = roc_auc_score(y_val, y_val_probs_svm)
print(f"✅ SVM Training Complete. Validation ROC-AUC: {auc_score:.4f}")

⏳ Training High-Fidelity SVM... (This uses CPU but is optimized)
✅ SVM Training Complete. Validation ROC-AUC: 0.7464


## 3. Deep Residual Neural Network (ResNet-MLP)

A standard MLP struggles when it gets too deep. We use **Residual Blocks** (skip connections) to allow gradients to flow easily, enabling us to use a deeper, more powerful architecture.

In [4]:
# --- Dataset Class ---
class LoanDataset(Dataset):
    def __init__(self, features, labels=None):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels.values, dtype=torch.float32).unsqueeze(1) if labels is not None else None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.features[idx], self.labels[idx]
        return self.features[idx]

# --- Residual Block Architecture ---
class ResBlock(nn.Module):
    def __init__(self, n_features, dropout_rate=0.3):
        super(ResBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Linear(n_features, n_features),
            nn.BatchNorm1d(n_features),
            nn.GELU(), # GELU is often better than ReLU for deep networks
            nn.Dropout(dropout_rate),
            nn.Linear(n_features, n_features),
            nn.BatchNorm1d(n_features)
        )
        self.activation = nn.GELU()

    def forward(self, x):
        identity = x
        out = self.block(x)
        out += identity  # Skip Connection
        return self.activation(out)

class DeepRiskNet(nn.Module):
    def __init__(self, input_dim):
        super(DeepRiskNet, self).__init__()
        
        # Initial projection
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.GELU()
        )
        
        # Stack of Residual Blocks
        self.res_blocks = nn.Sequential(
            ResBlock(512, dropout_rate=0.4),
            ResBlock(512, dropout_rate=0.4),
            nn.Linear(512, 256),
            nn.GELU(),
            ResBlock(256, dropout_rate=0.3),
            ResBlock(256, dropout_rate=0.3)
        )
        
        # Output Head
        self.output_head = nn.Sequential(
            nn.Linear(256, 64),
            nn.GELU(),
            nn.Linear(64, 1)
        )
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.input_layer(x)
        x = self.res_blocks(x)
        x = self.output_head(x)
        return self.sigmoid(x)

# --- Hyperparameters for RTX 4060 ---
BATCH_SIZE = 1024  # Large batch size for GPU efficiency
EPOCHS = 40
LEARNING_RATE = 0.005

# Loaders
train_dataset = LoanDataset(X_train, y_train)
val_dataset = LoanDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

# Model Init
model = DeepRiskNet(X_train.shape[1]).to(device)
criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

# OneCycle Scheduler (Super-convergence)
scheduler = OneCycleLR(optimizer, max_lr=LEARNING_RATE, steps_per_epoch=len(train_loader), epochs=EPOCHS)

# --- Training Loop ---
print("🚀 Starting Deep Learning Training...")
best_val_auc = 0
patience = 5
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    all_val_preds = []
    all_val_labels = []
    with torch.no_grad():
        for X_val_b, y_val_b in val_loader:
            X_val_b = X_val_b.to(device)
            preds = model(X_val_b)
            all_val_preds.extend(preds.cpu().numpy())
            all_val_labels.extend(y_val_b.numpy())
    
    val_auc = roc_auc_score(all_val_labels, all_val_preds)
    
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss/len(train_loader):.4f} | Val ROC-AUC: {val_auc:.4f}")
    
    # Early Stopping Check
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), 'best_model.pth')
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

# Load best model for inference
model.load_state_dict(torch.load('best_model.pth', weights_only=True))
print(f"🏆 Best Val AUC: {best_val_auc:.4f}")

🚀 Starting Deep Learning Training...
Epoch 1/40 | Train Loss: 0.3330 | Val ROC-AUC: 0.7517
Epoch 2/40 | Train Loss: 0.3168 | Val ROC-AUC: 0.7539
Epoch 3/40 | Train Loss: 0.3163 | Val ROC-AUC: 0.7527
Epoch 4/40 | Train Loss: 0.3161 | Val ROC-AUC: 0.7551
Epoch 5/40 | Train Loss: 0.3156 | Val ROC-AUC: 0.7533
Epoch 6/40 | Train Loss: 0.3155 | Val ROC-AUC: 0.7541
Epoch 7/40 | Train Loss: 0.3150 | Val ROC-AUC: 0.7546
Epoch 8/40 | Train Loss: 0.3150 | Val ROC-AUC: 0.7539
Epoch 9/40 | Train Loss: 0.3149 | Val ROC-AUC: 0.7529
⏹️ Early stopping triggered.
🏆 Best Val AUC: 0.7551


## 4. Generate Separate Submission Files

In [5]:
# --- 1. Generate Neural Network Predictions ---
test_tensor = torch.tensor(X_test_processed, dtype=torch.float32).to(device)
model.eval()
with torch.no_grad():
    nn_probs = model(test_tensor).cpu().numpy().flatten()
    nn_predictions = (nn_probs > 0.5).astype(int)

submission_nn = pd.DataFrame({
    'ProfileID': test_ids,
    'RiskFlag': nn_predictions
})
submission_nn.to_csv('submission_nn.csv', index=False)
print("✅ Generated 'submission_nn.csv' (Neural Network)")

# --- 2. Generate SVM Predictions ---
# Use the pipeline we trained earlier
svm_predictions = svm_pipeline.predict(X_test_processed)

submission_svm = pd.DataFrame({
    'ProfileID': test_ids,
    'RiskFlag': svm_predictions
})
submission_svm.to_csv('submission_svm.csv', index=False)
print("✅ Generated 'submission_svm.csv' (SVM)")

✅ Generated 'submission_nn.csv' (Neural Network)
✅ Generated 'submission_svm.csv' (SVM)
